# Pandas Tutorial: Importing Data with read_csv()

## Loading your CSV file into Python with Pandas

### The first step to any data science project is to import your data. Often, you'll work with data in Comma Separated Value (CSV) files. Learn how to use `pandas` to convert CSV files seemlessly into Python as `pandas` DataFrames, so that you can start your analysis right away.

Before we can use `pandas` to import our data, we need to know where our data file is in our filesystem and what our current working directory is. You'll see why this is important very soon, but let's review some basic concepts and Shell commands so that we can locate the data file. In this tutorial, we'll be importing nutrition data on 80 cereals into Python (Source: [Kaggle](https://www.kaggle.com/crawford/80-cereals)).

Everything on the computer is stored in the filesystem. 'Directories' is just another word for 'folders'. The working directory is simply the folder you're currently in. Here are some Shell commands you can use to navigate your way in the filesystem:

- The `pwd` command prints the path of your current working directory.
- The `ls` command lists all the files and sub-directories (i.e. directories in the current working directory).
- The `cd` command followed by the name of a sub-directory allows you to change your working directory to the sub-directory you specify.

Using these commands, can you locate the `cereal.csv` file?

------------------------------------------------------------------------------------------------------
*Insert DataCamp's Terminal console here for readers to interact with for early engagement*

*Like this but with Terminal console embedded if possible, if not, embed the IPython console and access file using `! ls` and `%cd` commands*
![](embedded_console.png)

------------------------------------------------------------------------------------------------------
Great! Now that we know what our current working directory is and where the data is in the filesystem, we can specify the file path to it. Let's import the CSV file into Python using `read_csv()` from `pandas`. Note that the `pandas` library is usually imported under the alias `pd`:

In [21]:
import pandas as pd
cereal_df = pd.read_csv("/Users/mm82089/dc/toots/data/cereal.csv")
cereal_df2 = pd.read_csv("data/cereal.csv")

# Are they the same?
print(pd.DataFrame.equals(cereal_df, cereal_df2))

True


As you can see in the code chunk above, the file path is the first argument to `read_csv()` and it was specified in two ways. You can use the full file path which includes the working directory or just use the relative file path. The `read_csv()` function is smart enough to decipher whether it's working with full or relative file paths and convert your flat file as a dataframe with ease.

In the following sections, we will see how else `pandas` increases the utility in importing CSV files. We'll learn some of the ways we can customize the `read_csv()` function, in the way that it deals with headers and missing data.

Headers refer to the column names of your dataset. For some datasets you might encounter, the headers may be completely missing, partially missing, or they may exist, but you may want to rename them. How do we deal with such issues effectively?

Let's take a closer look at our data:

In [22]:
import pandas as pd
df = pd.read_csv("data/cereal.csv")
print(df.head(5))

                         X.1      X.2      X.3       X.4      X.5      X.6  \
0                       name      mfr     type  calories  protein      fat   
1                  100% Bran        N        C        70        4        1   
2          100% Natural Bran        Q  no info       120        3        5   
3                   All-Bran  no info        C        70        4        1   
4  All-Bran with Extra Fiber        K        C        50        4  no info   

      X.7    X.8      X.9    X.10     X.11      X.12   X.13     X.14  X.15  \
0  sodium  fiber    carbo  sugars   potass  vitamins  shelf   weight  cups   
1     130     10  no info       6      280        25      3        1  0.33   
2      15      2        8       8      135         0      3        1     1   
3     260      9        7       5  no info        25      3        1  0.33   
4     140     14        8       0      330        25      3  no info   0.5   

        X.16  
0     rating  
1  68.402973  
2    no info  
3 

It seems like the actual column names are `name`, `mfr`, ..., `rating`, but it's incorrectly imported as the first observation in the dataset! Conveniently, the `read_csv()` function has an argument called `skiprows` that allows you to specify the line numbers to skip (note: it's 0-indexed), or the number of lines to skip at the start of the file. In this case, it seems like we want to skip the first line, so let's try importing our CSV file with `skiprows` set equal to 1:

In [23]:
df = pd.read_csv("data/cereal.csv", skiprows = 1)
print(df.head(5))

                        name      mfr     type  calories  protein      fat  \
0                  100% Bran        N        C        70        4        1   
1          100% Natural Bran        Q  no info       120        3        5   
2                   All-Bran  no info        C        70        4        1   
3  All-Bran with Extra Fiber        K        C        50        4  no info   
4             Almond Delight        R        C       110        2        2   

   sodium  fiber    carbo  sugars   potass  vitamins  shelf   weight  cups  \
0     130   10.0  no info       6      280        25      3        1  0.33   
1      15    2.0        8       8      135         0      3        1  1.00   
2     260    9.0        7       5  no info        25      3        1  0.33   
3     140   14.0        8       0      330        25      3  no info  0.50   
4     200    1.0       14       8       -1        25      3        1  0.75   

      rating  
0  68.402973  
1    no info  
2  59.425505  
3 

Nice! Even when we haven't specified the headers, the `read_csv()` function correctly infers that the first observation contains the headers for the dataset. Not only that, `read_csv()` infers the data types for each column as well. For example, the `calories` column is an integer column, where as the `fiber` column is a float column:

In [24]:
print(df['calories'].dtypes)
print(df['fiber'].dtypes)

int64
float64


By default, in `pandas`, columns with a string value are stored as type `object`. Because missing data here are encoded as `'no info'`, a string, if we check the data type for `fat`, a numeric column with missing values, we can see that its data type isn't ideal:

In [25]:
print(df['fat'].dtypes)

object


The `fat` column should be treated as type `int64`, and we want to recognize missing data as `NaN`. Instead of parsing through each column and replacing `'no info'` with `NaN` values after the data is loaded, we can customize the `read_csv()` function to take care of this before it's loaded using the `na_values` argument:

In [26]:
df = pd.read_csv("data/cereal.csv", skiprows = 1, na_values = 'no info')
print(df.head(5))

                        name  mfr type  calories  protein  fat  sodium  fiber  \
0                  100% Bran    N    C        70        4  1.0     130   10.0   
1          100% Natural Bran    Q  NaN       120        3  5.0      15    2.0   
2                   All-Bran  NaN    C        70        4  1.0     260    9.0   
3  All-Bran with Extra Fiber    K    C        50        4  NaN     140   14.0   
4             Almond Delight    R    C       110        2  2.0     200    1.0   

   carbo  sugars  potass  vitamins  shelf  weight  cups     rating  
0    NaN       6   280.0        25      3     1.0  0.33  68.402973  
1    8.0       8   135.0         0      3     1.0  1.00        NaN  
2    7.0       5     NaN        25      3     1.0  0.33  59.425505  
3    8.0       0   330.0        25      3     NaN  0.50  93.704912  
4   14.0       8    -1.0        25      3     1.0  0.75  34.384843  


Awesome. Now our data looks clean and all it took was one line of code with `pandas`'s `read_csv()` function.

## Final thoughts
There are other types of data files such as SAS, Excel, Stata you can import into Python using `pandas`. You can learn all the best practices of importing all kinds of data into Python in the [Importing Data in Python](https://www.datacamp.com/courses/importing-data-in-python-part-1) course series on DataCamp. Happy learning!

In [28]:
#help(pd.read_csv)